In [45]:
import numpy as np
import pandas as pd 
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
import time

from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
device

device(type='cpu')

# Introduction

#### In this notebook, I will be performing text classifcation using PyTorch. 
#### I hope this can be a good start to learn some basic NLP techniques.

# Data

In [95]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
sample = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

train_df = train_df.drop(["keyword", "location"], axis=1)
val_df = val_df.drop(["keyword", "location"], axis=1)
test_df = test.drop(["keyword", "location"], axis=1)

test_df['target'] = 0 # For the custom dataset later on

In [4]:
display(train_df)

,id,text,target
4996,7128,Courageous and honest analysis of need to use ...,1
3263,4688,@ZachZaidman @670TheScore wld b a shame if tha...,0
4907,6984,Tell @BarackObama to rescind medals of 'honor'...,1
2855,4103,Worried about how the CA drought might affect ...,1
4716,6706,@YoungHeroesID Lava Blast &amp; Power Red #Pan...,0
...,...,...,...
5226,7470,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,just had a panic attack bc I don't have enough...,0
860,1242,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,Officials say a quarantine is in place at an A...,1


#### The whole point of this exercise is to categorize the type of the text.
#### If the text is about a real disaster, it is labelled as 1.
#### If not, it is labelled as 0.

In [5]:
train_df.shape

(6090, 3)

#### We have 6090 total number of training data.

## Custom Dataset

#### Let's create a custom dataset.

In [96]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe        
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        
        X = self.df['text'].iloc[idx]
        y = torch.tensor(self.df['target'].iloc[idx], dtype=torch.float)
            
        return y, X

In [97]:
tokenizer = get_tokenizer('basic_english')

vocab_list = []
for row in tqdm(range(train_df.shape[0])):
    vocab_list.append(tokenizer(train_df['text'].iloc[row]))
    
vocab = build_vocab_from_iterator(vocab_list, specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

100%|██████████| 6090/6090 [00:00<00:00, 37184.09it/s]


In [98]:
vocab(['here', 'is', 'an', 'example'])

[126, 14, 55, 3380]

#### Working as intended!

In [99]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

## Custom DataLoader

In [100]:
train_dataset_custom = CustomDataset(train_df)
val_dataset_custom = CustomDataset(val_df)
test_dataset_custom = CustomDataset(test_df)

In [108]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_dataloader = DataLoader(train_dataset_custom, batch_size=8, shuffle=True, collate_fn=collate_batch)
val_dataloader = DataLoader(val_dataset_custom, batch_size=8, shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset_custom, batch_size=1, shuffle=False, collate_fn=collate_batch)

#### Before sending the data to our model, collate_fn function works on a batch of samples generated from DataLoader.
#### The offset is a tensor of delimiters to represent the beginning index of the individual sequence in the text tensor. 
#### Label is a tensor saving the labels of individual text entries.
#### Read more about it [here](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html).

# Modeling

In [102]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

# Training

In [103]:
num_class = 2
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [111]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

def submission(dataloader): # Code for submission
    preds = []
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            preds.append(predicted_label.argmax(1).item())
            
    return preds

In [105]:
EPOCHS = 10
LR = 5

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(val_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)
    print("\n")

| epoch   1 |   500/  762 batches | accuracy    0.653
-----------------------------------------------------------
| end of epoch   1 | time:  2.72s | valid accuracy    0.731 
-----------------------------------------------------------


| epoch   2 |   500/  762 batches | accuracy    0.784
-----------------------------------------------------------
| end of epoch   2 | time:  2.59s | valid accuracy    0.760 
-----------------------------------------------------------


| epoch   3 |   500/  762 batches | accuracy    0.827
-----------------------------------------------------------
| end of epoch   3 | time:  2.78s | valid accuracy    0.787 
-----------------------------------------------------------


| epoch   4 |   500/  762 batches | accuracy    0.861
-----------------------------------------------------------
| end of epoch   4 | time:  2.68s | valid accuracy    0.749 
-----------------------------------------------------------


| epoch   5 |   500/  762 batches | accuracy    0.90

# Submission

In [115]:
preds = submission(test_dataloader)

sample['target'] = preds

sample.to_csv('submission.csv', index=False)